In [2]:
import os 
import cv2 
import random 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard 
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
import hashlib

# Preprocessing

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset',
    batch_size=32,
    image_size=(256, 256),  # Target size for all images
    validation_split=0.2,
    subset='training',
    seed=123,
    label_mode='categorical'
)

# Function to check the shape of images in the dataset
def check_shape(images, labels):
    print("Image batch shape:", images.shape)  # Print the shape of the image batch
    return images, labels

Found 4217 files belonging to 4 classes.
Using 3374 files for training.


#### Rescaling

In [4]:
rescale = tf.keras.layers.Rescaling(1./255)

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset',
    batch_size = 32,
    image_size = (256, 256),
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    label_mode = 'categorical'
)

train_ds = train_ds.map(lambda x, y: (rescale(x), y))

validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset', 
    batch_size = 32, 
    image_size = (256, 256), 
    validation_split = 0.2, 
    subset = 'validation', 
    seed = 123, 
    label_mode = 'categorical'
)

validation_ds = validation_ds.map(lambda x, y: (rescale(x), y))

test_ds = tf.keras.utils.image_dataset_from_directory(
    directory='D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset',
    batch_size=32, 
    image_size=(256, 256), 
    label_mode='categorical', 
    shuffle=False, 
)

test_ds = test_ds.map(lambda x, y: (rescale(x), y))

Found 4217 files belonging to 4 classes.
Using 3374 files for training.
Found 4217 files belonging to 4 classes.
Using 843 files for validation.
Found 4217 files belonging to 4 classes.


#### Cek shape after preprocessing

In [5]:
#dataset train, validation, dan tes sudah diubah ukurannya menjadi 256x256 piksel dan memiliki 3 channel (format RGB)

print('Shape of the first image in the training dataset:', next(iter(train_ds))[0][0].shape)
print('Shape of the first iage in the validation dataset:', next(iter(validation_ds))[0][0].shape)
print('Shape of the first image in the test dataset:', next(iter(test_ds))[0][0].shape)

Shape of the first image in the training dataset: (256, 256, 3)
Shape of the first iage in the validation dataset: (256, 256, 3)
Shape of the first image in the test dataset: (256, 256, 3)


#### Check pixel value after preprocessing

In [6]:
min_pixel_value = float('inf')
max_pixel_value = float('-inf')

for image, _ in train_ds: 
    batch_min = tf.reduce_min(image)
    batch_max = tf.reduce_max(image)

    min_pixel_value = tf.minimum(min_pixel_value, batch_min)
    max_pixel_value = tf.maximum(max_pixel_value, batch_max)

print('Minimum pixel value:', max_pixel_value.numpy())
print('Maaximum pixel value:', max_pixel_value.numpy())

Minimum pixel value: 1.0
Maaximum pixel value: 1.0


In [7]:
import os
import hashlib
from PIL import Image

def generate_image_hash(image_path):
    """Menghasilkan hash untuk gambar"""
    try:
        with Image.open(image_path) as img:
            # Konversi gambar ke ukuran yang sama untuk membuat perbandingan lebih mudah
            img = img.resize((256, 256))
            img_hash = hashlib.md5(img.tobytes()).hexdigest()
        return img_hash
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def find_duplicate_images(folder_path):
    """Mencari gambar duplikat dalam folder"""
    hashes = {}
    duplicates = []

    # Iterasi melalui semua file dalam folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Cek apakah file adalah gambar
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_hash = generate_image_hash(file_path)

            # Lanjutkan jika hash berhasil dihasilkan
            if img_hash:
                # Cek apakah hash sudah ada di kamus
                if img_hash in hashes:
                    duplicates.append((file_path, hashes[img_hash]))
                else:
                    hashes[img_hash] = file_path

    return duplicates

# Penggunaan
folder_path = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset'  # Ganti dengan path folder Anda
dups = find_duplicate_images(folder_path)

if dups:
    print("Gambar duplikat ditemukan:")
    for dup in dups:
        print(f"Duplikat: {dup[0]} adalah salinan dari {dup[1]}")
else:
    print("Tidak ada gambar duplikat ditemukan.")


Tidak ada gambar duplikat ditemukan.


#### Grayscale

##### grayscale cataract

In [14]:
input_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset/cataract'
output_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/grayscale/cataract'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):

        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, gray_img)

print('Proses selesai, citra grayscaale disimpan di folder output. ')

Proses selesai, citra grayscaale disimpan di folder output. 


##### gryascale diabetic_retinopathy

In [16]:
input_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset/diabetic_retinopathy'
output_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/grayscale/diabetic_retinopathy'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):

        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, gray_img)

print('Proses selesai, citra grayscaale disimpan di folder output. ')

Proses selesai, citra grayscaale disimpan di folder output. 


##### grayscale glaucoma

In [17]:
input_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset/glaucoma'
output_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/grayscale/glaucoma'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):

        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, gray_img)

print('Proses selesai, citra grayscaale disimpan di folder output. ')

Proses selesai, citra grayscaale disimpan di folder output. 


##### grayscale normal

In [18]:
input_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/dataset/normal'
output_folder = 'D:/Kegiatan lain/Stechoq/dataset tugas akhir/grayscale/normal'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):

        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, gray_img)

print('Proses selesai, citra grayscaale disimpan di folder output. ')

Proses selesai, citra grayscaale disimpan di folder output. 
